# Notebook to generate n% sampled data in a schema from population schema

In [ ]:
# Import necessary libraries
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pyspark.sql.functions as F
from pyspark.sql.window import Window

## Configurable params for the notebook are:

- `input_schema` defines the schema to be used to sample out data from and is 100% of the data as per prod runs
- `output_schema` defines the schema to be used to store the sampled data suffixed by `_n_percent` where n is any number.
- `fraction value` defines the fraction of data to be sampled from the complete dataset.
- `join_id` defines the name of the column based on which filtering and querying of the data is to be done

In [ ]:
# Configurable params for the notebook
input_schema = "unifai_stable"
output_schema = "sampled"
fraction_value = 0.10
join_id = 'member_id'

## Create Schema to store n% data from input schema

In [ ]:
# Create schema in Hive metastore if it does not exist
fraction_percent = str(int(fraction_value*100))
spark.sql(f'CREATE SCHEMA IF NOT EXISTS {output_schema}_{fraction_percent}_percent')

Out[3]: DataFrame[]

## Select random records from input schema and filter out data from it

In [ ]:
# Using TABLESAMPLE
final_ids = spark.sql(f"SELECT DISTINCT {join_id} FROM {input_schema}.eods_elig_risk TABLESAMPLE ({fraction_percent} PERCENT) REPEATABLE (123)")
# persisting for look up logic
final_ids.persist()

## Get list of tables from input schema from which data is to be sampled

In [ ]:
# Get list of tables in input schema tables
table_list = [name[0] for name in spark.catalog.listTables(f"{input_schema}")]
# Separate logic required for rxclaims table. Will not be applying the common logic.
table_list.remove('eods_rxclaims')

# Partition Columns Map
partition_col_map = {'eods_elig_risk': ('member_id', 'CVG_MONTH'),
      'eods_member_attrib': ('member_id', 'ANCHOR_MONTH')}

## Loop to iterate through input schema tables and create sampled data into output schema suffixed by _n_percent

In [ ]:
# Loop to iterate over tables and write sampled percentage of data to the output schema

for table_name in table_list:
  # Load and filter data
  print(f"Writing data to {table_name}")
  if table_name in partition_col_map:
    partition_col = partition_col_map[table_name]
  else:
    partition_col = ('member_id',)
  input_table = spark.sql(f"SELECT * FROM {input_schema}.{table_name}")
  sampled_table = input_table.join(final_ids, on='member_id') 
  if 'ROW_ID' not in sampled_table.columns:
   sampled_table = sampled_table.withColumn('ROW_ID', F.monotonically_increasing_id())
  # De-Duplication using partition by on member_id and using deterministic ROW_ID column 
  deduped_sampled_table = sampled_table.withColumn('row_number', F.row_number().over(Window.partitionBy(*partition_col).orderBy('ROW_ID'))) \
                   .filter(F.col('row_number') == 1) \
                   .drop('row_number')
  # Write data
  deduped_sampled_table.write.format("delta").saveAsTable(f"{output_schema}_{fraction_percent}_percent.{table_name}")
  print(f"Finished writing data to {table_name}")
  # Report
  input_table_count = input_table.count()
  input_id_count = input_table.select(join_id).drop_duplicates().count()
  deduped_sampled_table_count = deduped_sampled_table.count()
  deduped_sampled_id_count = deduped_sampled_table.select(join_id).drop_duplicates().count()

  print(f'{input_table_count:,} -> {deduped_sampled_table_count:,} records ({deduped_sampled_table_count/input_table_count:.2%} records sample)')
  print(f'{input_id_count:,} -> {deduped_sampled_id_count:,} UNIQUE IDs ({deduped_sampled_id_count/input_id_count:.2%} UNIQUE ID sample)')

Writing data to cdf_ce_table
Finished writing data to cdf_ce_table
11,177,328 -> 1,118,039 records (10.00% records sample)
9,808,658 -> 980,865 UNIQUE IDs (10.00% UNIQUE ID sample)
Writing data to cdf_code_table
Finished writing data to cdf_code_table
4,677,450,475 -> 432,283,184 records (9.24% records sample)
10,499,622 -> 834,647 UNIQUE IDs (7.95% UNIQUE ID sample)
Writing data to cdf_code_table_new_ancillary_ckd
Finished writing data to cdf_code_table_new_ancillary_ckd
4,539,738,745 -> 418,572,516 records (9.22% records sample)
10,499,749 -> 834,210 UNIQUE IDs (7.95% UNIQUE ID sample)
Writing data to cdf_lab_table
Finished writing data to cdf_lab_table
850,882,133 -> 77,198,314 records (9.07% records sample)
5,031,278 -> 414,914 UNIQUE IDs (8.25% UNIQUE ID sample)
Writing data to cdf_member_table
Finished writing data to cdf_member_table
10,846,687 -> 1,084,072 records (9.99% records sample)
9,808,658 -> 980,865 UNIQUE IDs (10.00% UNIQUE ID sample)
Writing data to cdf_util_table
Finished writing data to cdf_util_table
17,881,358 -> 1,690,529 records (9.45% records sample)
4,266,084 -> 395,804 UNIQUE IDs (9.28% UNIQUE ID sample)
Writing data to eods_admits
Finished writing data to eods_admits
4,284,748 -> 391,647 records (9.14% records sample)
1,736,743 -> 157,030 UNIQUE IDs (9.04% UNIQUE ID sample)
Writing data to eods_covid19_ip_cov_test
Finished writing data to eods_covid19_ip_cov_test
1,709,326 -> 153,124 records (8.96% records sample)
814,228 -> 74,438 UNIQUE IDs (9.14% UNIQUE ID sample)
Writing data to eods_dod
Finished writing data to eods_dod
493,920 -> 44,708 records (9.05% records sample)
493,917 -> 44,733 UNIQUE IDs (9.06% UNIQUE ID sample)
Writing data to eods_elig_risk
Finished writing data to eods_elig_risk
238,510,913 -> 23,877,352 records (10.01% records sample)
9,808,658 -> 980,865 UNIQUE IDs (10.00% UNIQUE ID sample)
Writing data to eods_eods_mbr_demographic_orec
Finished writing data to eods_eods_mbr_demographic_orec
10,244,459 -> 932,421 records (9.10% records sample)
10,232,168 -> 931,715 UNIQUE IDs (9.11% UNIQUE ID sample)
Writing data to eods_er_uc
Finished writing data to eods_er_uc
7,744,863 -> 732,910 records (9.46% records sample)
2,671,404 -> 250,376 UNIQUE IDs (9.37% UNIQUE ID sample)
Writing data to eods_labresults
Finished writing data to eods_labresults
2,021,630,926 -> 184,486,683 records (9.13% records sample)
6,077,462 -> 490,573 UNIQUE IDs (8.07% UNIQUE ID sample)
Writing data to eods_medclaims
Finished writing data to eods_medclaims
1,586,883,412 -> 145,590,839 records (9.17% records sample)
8,974,913 -> 766,662 UNIQUE IDs (8.54% UNIQUE ID sample)
Writing data to eods_member_attrib
Finished writing data to eods_member_attrib

In [ ]:
# Pharmacy - RxClaims table de-dupe
# Load and filter data
table_name = 'eods_rxclaims'
print(f"Writing data to {table_name}")
input_table = spark.sql(f"SELECT * FROM {input_schema}.{table_name}")
sampled_table = input_table.join(final_ids, on='member_id') 
# De-Duplication - specific to rxclaims table
target_cond = '(CLM_STS != "06" OR CLM_STS IS NULL) AND DAY_SPL >= 28 and DAY_SPL < 84 AND GPI_2_CLASS LIKE "%OPIOID%"'
clean_rows = sampled_table.filter(~F.expr(target_cond))
updated_rows = sampled_table.filter(F.expr(target_cond)) \
                .withColumn('row_number', F.row_number().over(Window.partitionBy('member_id', 'GPI_6_CLASS', 'DT_OF_SRVC').orderBy('MME', 'ROW_ID'))) \
                .filter(F.col('row_number') == 1) \
                .drop('row_number')
deduped_sampled_table = clean_rows.unionByName(updated_rows)
# Write data
deduped_sampled_table.write.format("delta").saveAsTable(f"{output_schema}_{fraction_percent}_percent.{table_name}")
print(f"Finished writing data to {table_name}")
# Report
input_table_count = input_table.count()
input_id_count = input_table.select(join_id).drop_duplicates().count()
deduped_sampled_table_count = deduped_sampled_table.count()
deduped_sampled_id_count = deduped_sampled_table.select(join_id).drop_duplicates().count()

print(f'{input_table_count:,} -> {deduped_sampled_table_count:,} records ({deduped_sampled_table_count/input_table_count:.2%} records sample)')
print(f'{input_id_count:,} -> {deduped_sampled_id_count:,} UNIQUE IDs ({deduped_sampled_id_count/input_id_count:.2%} UNIQUE ID sample)')

final_ids.unpersist(blocking=True)

## Unit test cases

### Test case 1: Check if all the member id's are present in sampled and input schema for a table

In [ ]:
from pyspark.sql.functions import *
input_table = spark.sql(f"SELECT * FROM {input_schema}.cdf_ce_table")
sampled_table = spark.sql(f"SELECT * FROM sampled_{fraction_percent}_percent.cdf_ce_table")
input_table.groupBy("member_id").count().where("count = 8").filter(col('member_id')== '8369268-01|10006')
sampled_table.groupBy("member_id").count().where("count = 8").filter(col('member_id')== '8369268-01|10006')

Out[5]: DataFrame[member_id: string, count: bigint]

### Test case 2: Check if there are any null values in sampled dataframe

In [ ]:
sampled_table.filter(col("member_id").isNull())

+---------+----------------+-------------+-----------+
member_id|line_of_business|ce_start_date|ce_end_date|
+---------+----------------+-------------+-----------+
+---------+----------------+-------------+-----------+

### Test case 3: Check if schema is the same

In [ ]:
for idx1,el1 in enumerate(input_table.dtypes):
    for idx2,el2 in enumerate(sampled_table.dtypes):
        if idx1 == idx2:
            if el1[0] == el2[0] and el1[1] == el2[1]:
                continue
            else:
                raise ValueError("Schema Don't Match for Col {0} and {1}".format(el1[0],el2[0]))


### Test case 4: Check if input schema executed successfully for all the tables from input schema

In [ ]:
input_table_len = set([name[0] for name in spark.catalog.listTables(f"{input_schema}")])
sampled_table_len = set([name[0] for name in spark.catalog.listTables(f"{output_schema}_{fraction_percent}_percent")])
diff_table = input_table_len - sampled_table_len
if len(diff_table) == 0:
  print("All tables sampled")
else:
  print(f"Check for errors. {diff_table} tables are not sampled correctly")

Check for errors. {'eods_obsstays', 'eods_rxclaims', 'falls_combined_features'} tables are not sampled correctly